## Modeling for predicting hospitalization at ED triage

Work Flow:
1. Task-specific filter
2. Variable selection
3. Modeling script
4. Performance output


### Load train and test

In [ ]:
import os
import time
import random
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from helpers import PlotROCCurve

from dataset_path import output_path

path = output_path
output_path = os.path.join(path, "Figure3")
df_train = pd.read_csv((os.path.join(path, 'train.csv')))
df_test = pd.read_csv((os.path.join(path, 'test.csv')))

train_cxr_note_embs = list(np.load((os.path.join(path, 'train_cxr_note_embs.npy'))))
test_cxr_note_embs = list(np.load((os.path.join(path, 'test_cxr_note_embs.npy'))))

train_cxr_note_embs = pd.DataFrame(train_cxr_note_embs, columns = ['cxr_note_emb'+str(i) for i in range(128)])
test_cxr_note_embs = pd.DataFrame(test_cxr_note_embs, columns = ['cxr_note_emb'+str(i) for i in range(128)])


df_train = pd.concat([df_train, train_cxr_note_embs], axis = 1)
df_test = pd.concat([df_test, test_cxr_note_embs], axis = 1)
confidence_interval = 95
random_seed=0

In [ ]:
random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_rows', 100) 
df_train.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age,anchor_year,dod,admittime,dischtime,deathtime,ethnicity,edregtime,edouttime,insurance,in_year,age,outcome_inhospital_mortality,ed_los,intime_icu,time_to_icu_transfer,outcome_icu_transfer_12h,outcome_hospitalization,outcome_critical,n_ed_30d,n_ed_90d,n_ed_365d,next_ed_visit_time,next_ed_visit_time_diff,outcome_ed_revisit_3d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,ed_los_hours,time_to_icu_transfer_hours,next_ed_visit_time_diff_days,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,...,cxr_note_emb78,cxr_note_emb79,cxr_note_emb80,cxr_note_emb81,cxr_note_emb82,cxr_note_emb83,cxr_note_emb84,cxr_note_emb85,cxr_note_emb86,cxr_note_emb87,cxr_note_emb88,cxr_note_emb89,cxr_note_emb90,cxr_note_emb91,cxr_note_emb92,cxr_note_emb93,cxr_note_emb94,cxr_note_emb95,cxr_note_emb96,cxr_note_emb97,cxr_note_emb98,cxr_note_emb99,cxr_note_emb100,cxr_note_emb101,cxr_note_emb102,cxr_note_emb103,cxr_note_emb104,cxr_note_emb105,cxr_note_emb106,cxr_note_emb107,cxr_note_emb108,cxr_note_emb109,cxr_note_emb110,cxr_note_emb111,cxr_note_emb112,cxr_note_emb113,cxr_note_emb114,cxr_note_emb115,cxr_note_emb116,cxr_note_emb117,cxr_note_emb118,cxr_note_emb119,cxr_note_emb120,cxr_note_emb121,cxr_note_emb122,cxr_note_emb123,cxr_note_emb124,cxr_note_emb125,cxr_note_emb126,cxr_note_emb127
0,183244,14328106,NaN,32336955,2159-11-08 15:13:00,2159-11-09 01:47:00,F,WHITE,WALK IN,HOME,75,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,75,False,0 days 10:34:00,NaN,NaN,False,False,False,0,0,0,NaN,NaN,False,0,0,0,0,0,0,10.57,NaN,NaN,38.055556,65.0,18.0,100.0,135.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,376935,18860711,NaN,35614382,2159-10-24 06:19:00,2159-10-24 14:31:00,F,WHITE,WALK IN,HOME,28,2159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159,28,False,0 days 08:12:00,NaN,NaN,False,False,False,1,1,1,NaN,NaN,False,0,0,0,0,0,0,8.20,NaN,NaN,36.500000,89.0,18.0,97.0,103.0,46.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,291579,16860825,NaN,37485359,2203-01-07 17:45:00,2203-01-08 07:00:00,F,BLACK/AFRICAN AMERICAN,AMBULANCE,HOME,52,2194,NaN,NaN,NaN,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NaN,2203,61,False,0 days 13:15:00,NaN,NaN,False,False,False,0,0,5,2203-01-12 22:26:00,4 days 15:26:00,False,0,0,2,0,0,0,13.25,NaN,4.64,36.888889,96.0,16.0,96.0,160.0,80.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,210524,14971343,29116455.0,34783406,2207-07-11 18:24:00,2207-07-11 22:37:00,F,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED,74,2198,NaN,2207-07-11 20:24:00,2207-07-12 19:15:00,NaN,BLACK/AFRICAN AMERICAN,2207-07-11 18:24:00,2207-07-11 22:37:00,Medicare,2207,83,False,0 days 04:13:00,NaN,NaN,False,True,False,0,0,0,2208-08-28 08:10:00,413 days 09:33:00,False,0,0,0,0,0,0,4.22,NaN,413.40,36.944444,104.0,16.0,100.0,172.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,69316,11648387,NaN,39707189,2181-01-23 09:25:00,2181-01-23 13:27:00,M,WHITE,WALK IN,HOME,64,2175,2183-08-25,NaN,NaN,NaN,WHITE,NaN,NaN,NaN,2181,70,False,0 days 04:02:00,NaN,NaN,False,False,False,0,7,19,2181-01-27 13:02:00,3 days 23:35:00,False,0,2,4,0,0,0,4.03,NaN,3.98,36.333333,118.0,18.0,98.0,166.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### 1. Task-specific filter: No task-specific exclusion needed

In [ ]:
print('training size =', len(df_train), ', testing size =', len(df_test))

training size = 334480 , testing size = 83620


### 2. Variable Selection

In [ ]:
variable = ["age", "gender", 
            
            "n_ed_30d", "n_ed_90d", "n_ed_365d", "n_hosp_30d", "n_hosp_90d", 
            "n_hosp_365d", "n_icu_30d", "n_icu_90d", "n_icu_365d", 
            
            "triage_temperature", "triage_heartrate", "triage_resprate", 
            "triage_o2sat", "triage_sbp", "triage_dbp", "triage_pain", "triage_acuity",
            
            "chiefcom_chest_pain", "chiefcom_abdominal_pain", "chiefcom_headache",
            "chiefcom_shortness_of_breath", "chiefcom_back_pain", "chiefcom_cough", 
            "chiefcom_nausea_vomiting", "chiefcom_fever_chills", "chiefcom_syncope", 
            "chiefcom_dizziness", 
            
            "cci_MI", "cci_CHF", "cci_PVD", "cci_Stroke", "cci_Dementia", 
            "cci_Pulmonary", "cci_Rheumatic", "cci_PUD", "cci_Liver1", "cci_DM1", 
            "cci_DM2", "cci_Paralysis", "cci_Renal", "cci_Cancer1", "cci_Liver2", 
            "cci_Cancer2", "cci_HIV", 
            
            "eci_Arrhythmia", "eci_Valvular", "eci_PHTN",  "eci_HTN1", "eci_HTN2", 
            "eci_NeuroOther", "eci_Hypothyroid", "eci_Lymphoma", "eci_Coagulopathy", 
            "eci_Obesity", "eci_WeightLoss", "eci_FluidsLytes", "eci_BloodLoss",
            "eci_Anemia", "eci_Alcohol", "eci_Drugs","eci_Psychoses", "eci_Depression"]
variable.extend(list(test_cxr_note_embs.columns))
print(variable)
outcome = "outcome_hospitalization"

['age', 'gender', 'n_ed_30d', 'n_ed_90d', 'n_ed_365d', 'n_hosp_30d', 'n_hosp_90d', 'n_hosp_365d', 'n_icu_30d', 'n_icu_90d', 'n_icu_365d', 'triage_temperature', 'triage_heartrate', 'triage_resprate', 'triage_o2sat', 'triage_sbp', 'triage_dbp', 'triage_pain', 'triage_acuity', 'chiefcom_chest_pain', 'chiefcom_abdominal_pain', 'chiefcom_headache', 'chiefcom_shortness_of_breath', 'chiefcom_back_pain', 'chiefcom_cough', 'chiefcom_nausea_vomiting', 'chiefcom_fever_chills', 'chiefcom_syncope', 'chiefcom_dizziness', 'cci_MI', 'cci_CHF', 'cci_PVD', 'cci_Stroke', 'cci_Dementia', 'cci_Pulmonary', 'cci_Rheumatic', 'cci_PUD', 'cci_Liver1', 'cci_DM1', 'cci_DM2', 'cci_Paralysis', 'cci_Renal', 'cci_Cancer1', 'cci_Liver2', 'cci_Cancer2', 'cci_HIV', 'eci_Arrhythmia', 'eci_Valvular', 'eci_PHTN', 'eci_HTN1', 'eci_HTN2', 'eci_NeuroOther', 'eci_Hypothyroid', 'eci_Lymphoma', 'eci_Coagulopathy', 'eci_Obesity', 'eci_WeightLoss', 'eci_FluidsLytes', 'eci_BloodLoss', 'eci_Anemia', 'eci_Alcohol', 'eci_Drugs', 'eci_

In [ ]:
X_train = df_train[variable].copy()
y_train = df_train[outcome].copy()
X_test = df_test[variable].copy()
y_test = df_test[outcome].copy()

In [ ]:
X_train.dtypes.to_frame().T

,age,gender,n_ed_30d,n_ed_90d,n_ed_365d,n_hosp_30d,n_hosp_90d,n_hosp_365d,n_icu_30d,n_icu_90d,n_icu_365d,triage_temperature,triage_heartrate,triage_resprate,triage_o2sat,triage_sbp,triage_dbp,triage_pain,triage_acuity,chiefcom_chest_pain,chiefcom_abdominal_pain,chiefcom_headache,chiefcom_shortness_of_breath,chiefcom_back_pain,chiefcom_cough,chiefcom_nausea_vomiting,chiefcom_fever_chills,chiefcom_syncope,chiefcom_dizziness,cci_MI,cci_CHF,cci_PVD,cci_Stroke,cci_Dementia,cci_Pulmonary,cci_Rheumatic,cci_PUD,cci_Liver1,cci_DM1,cci_DM2,cci_Paralysis,cci_Renal,cci_Cancer1,cci_Liver2,cci_Cancer2,cci_HIV,eci_Arrhythmia,eci_Valvular,eci_PHTN,eci_HTN1,...,cxr_note_emb78,cxr_note_emb79,cxr_note_emb80,cxr_note_emb81,cxr_note_emb82,cxr_note_emb83,cxr_note_emb84,cxr_note_emb85,cxr_note_emb86,cxr_note_emb87,cxr_note_emb88,cxr_note_emb89,cxr_note_emb90,cxr_note_emb91,cxr_note_emb92,cxr_note_emb93,cxr_note_emb94,cxr_note_emb95,cxr_note_emb96,cxr_note_emb97,cxr_note_emb98,cxr_note_emb99,cxr_note_emb100,cxr_note_emb101,cxr_note_emb102,cxr_note_emb103,cxr_note_emb104,cxr_note_emb105,cxr_note_emb106,cxr_note_emb107,cxr_note_emb108,cxr_note_emb109,cxr_note_emb110,cxr_note_emb111,cxr_note_emb112,cxr_note_emb113,cxr_note_emb114,cxr_note_emb115,cxr_note_emb116,cxr_note_emb117,cxr_note_emb118,cxr_note_emb119,cxr_note_emb120,cxr_note_emb121,cxr_note_emb122,cxr_note_emb123,cxr_note_emb124,cxr_note_emb125,cxr_note_emb126,cxr_note_emb127
0,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [ ]:
encoder = LabelEncoder()
X_train['gender'] = encoder.fit_transform(X_train['gender'])
X_test['gender'] = encoder.transform(X_test['gender'])

In [ ]:
print('class ratio')
ratio = y_train.sum()/(~y_train).sum()
print('positive : negative =', ratio, ': 1')

class ratio
positive : negative = 0.8981573436693093 : 1


### 3.  Modeling script

In [ ]:
# Containers for all results
result_list = []

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.dense_1 = Dense(200, activation='relu')
        self.dense_2 = Dense(64, activation='relu')
        self.dense_3 = Dense(10, activation='relu')
        self.classifier = Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return self.classifier(x)

In [ ]:
# skip this cell if not retraining
mlp = MLP()
mlp.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(learning_rate=0.001), 
              metrics=['accuracy', 'AUC', {'auprc': metrics.AUC(name='auprc', curve='PR')}, 
                       'TruePositives', 'TrueNegatives', 'Precision', 'Recall'])
start = time.time()
mlp.fit(X_train.astype(np.float32), y_train, batch_size=200, epochs=200)
runtime = time.time() - start
print('Training time:', runtime, 'seconds')
mlp.save('hospitalization_triage_mlp')

Epoch 1/200
1673/1673 [==============================] - 4s 2ms/step - loss: 0.5719 - accuracy: 0.7128 - auc: 0.7813 - auprc: 0.7437 - true_positives: 107026.0000 - true_negatives: 131408.0000 - precision: 0.7049 - recall: 0.6762 0s - loss: 0.5843 - accuracy: 0.7066 - auc: 0.7728 - auprc: 0.7326 - true_positives: 76405.0000 - true_negatives: 95855.0000 - precision: 
Epoch 2/200
1673/1673 [==============================] - 3s 2ms/step - loss: 0.5322 - accuracy: 0.7318 - auc: 0.8082 - auprc: 0.7786 - true_positives: 113467.0000 - true_negatives: 131295.0000 - precision: 0.7164 - recall: 0.7169
Epoch 3/200
1673/1673 [==============================] - 3s 2ms/step - loss: 0.5282 - accuracy: 0.7347 - auc: 0.8114 - auprc: 0.7830 - true_positives: 114329.0000 - true_negatives: 131425.0000 - precision: 0.7185 - recall: 0.7224
Epoch 4/200
1673/1673 [==============================] - 4s 2ms/step - loss: 0.5253 - accuracy: 0.7372 - auc: 0.8138 - auprc: 0.7863 - true_positives: 115198.0000 - true_n